In [1]:
import pandas as pd
import geopandas as gpd
import datetime
import os
import shapely


In [2]:
def abrir_dados_estacoes() -> gpd.GeoDataFrame:     
    dados_inmet = {
        'regiao': [],
        'estado': [],
        'codigo_estacao': [],
        'nome_estacao': [],
        'lat': [],
        'long': [],
        'alt': [],
        'data_fundacao': [],
        'nome_arquivo': []
    }
    caminho_inmet = './data/INMET'
    for dir in os.listdir(caminho_inmet):
        if os.path.isdir(os.path.join(caminho_inmet, dir)):
            for file in os.listdir(os.path.join(caminho_inmet, dir)):
                if file.endswith('.csv') or file.endswith('.CSV'):
                    f = open(os.path.join(caminho_inmet, dir, file), 'r', encoding='latin1')
                    lines = [f.readline().split(';')[-1].strip() for _ in range(8)]
                    dados_inmet['regiao'].append(lines[0])
                    dados_inmet['estado'].append(lines[1])
                    dados_inmet['codigo_estacao'].append(lines[2])
                    dados_inmet['nome_estacao'].append(lines[3])
                    dados_inmet['lat'].append(float(lines[4].replace(',', '.')))
                    dados_inmet['long'].append(float(lines[5].replace(',', '.')))
                    dados_inmet['alt'].append(float(lines[6].replace(',', '.')))
                    dados_inmet['data_fundacao'].append(datetime.datetime.strptime(lines[7], '%d/%m/%y'))
                    dados_inmet['nome_arquivo'].append(os.path.join(caminho_inmet, dir, file))

    print("[info] Dados INMET carregados")
    return gpd.GeoDataFrame(dados_inmet, 
                            geometry=gpd.points_from_xy(dados_inmet['long'], dados_inmet['lat'])
                            ).drop(columns=['lat', 'long'])

In [3]:
def df2gdf_linestring(df: pd.DataFrame, ignora_sentido: bool) -> gpd.GeoDataFrame:
    df.sort_values(['rodovia', 'km_m', 'sentido'], inplace=True, ignore_index=True)
    df_return = {
        "rodovia": [],
        "sentido": [],
        "concessionaria": [],
        "km": [],
        "ano_do_pnv_snv": [],
        "geometry": []
    }
    pontos = []

    linha_anterior = df.iloc[0]
    for i, linha in df.iterrows():
        if linha['rodovia'] != linha_anterior['rodovia'] or (linha['sentido'] != linha_anterior['sentido'] and not ignora_sentido):
            df_return["rodovia"].append(linha_anterior['rodovia'])
            df_return["sentido"].append(linha_anterior['sentido'])
            df_return["concessionaria"].append(linha_anterior['concessionaria'])
            df_return["km"].append(df[(df['rodovia'] == linha_anterior['rodovia']) &
                                  (df['sentido'] == linha_anterior['sentido']) &
                                  (df['concessionaria'] == linha_anterior['concessionaria'])]['km_m'].max())
            df_return["ano_do_pnv_snv"].append(linha_anterior['ano_do_pnv_snv'])
            if len(pontos) > 1:
                df_return["geometry"].append(shapely.geometry.LineString(pontos))
            else:
                pontos.append((linha_anterior['longitude'], linha_anterior['latitude']))
                df_return["geometry"].append(shapely.geometry.LineString(pontos))
            pontos = []
            
        pontos.append((linha['longitude'], linha['latitude']))
        linha_anterior = linha


    return gpd.GeoDataFrame(df_return, crs="EPSG:4326")

In [4]:
inmet_gdf = abrir_dados_estacoes()
inmet_gdf.crs = "EPSG:4326"
print(inmet_gdf.head())
print(inmet_gdf.info())

[info] Dados INMET carregados
  regiao estado                    codigo_estacao nome_estacao     alt  \
0     NE     PE                           SURUBIM         A328  421.44   
1      N     TO                          MATEIROS         A040  791.00   
2     SE     RJ  CAMPOS DOS GOYTACAZES - SAO TOME         A620    7.00   
3     NE     PB                             PATOS         A321  263.66   
4     CO     MT                PRIMAVERA DO LESTE         A923  680.00   

  data_fundacao                                       nome_arquivo  \
0    2008-03-09  ./data/INMET/2024/INMET_NE_PE_A328_SURUBIM_01-...   
1    2012-10-25  ./data/INMET/2024/INMET_N_TO_A040_MATEIROS_01-...   
2    2008-06-13  ./data/INMET/2024/INMET_SE_RJ_A620_CAMPOS DOS ...   
3    2007-07-21  ./data/INMET/2024/INMET_NE_PB_A321_PATOS_01-01...   
4    2017-10-26  ./data/INMET/2024/INMET_CO_MT_A923_PRIMAVERA D...   

                      geometry  
0   POINT (-35.80111 -7.83972)  
1  POINT (-45.92194 -10.43444)  
2  PO

In [5]:
pista_principal_df = pd.read_csv('data/ANTT/km_pista_principal/dados_dos_quilometro_principal.csv', sep=';', encoding='latin1')
pista_principal_df['latitude'] = pista_principal_df['latitude'].str.replace(',', '.').astype(float)
pista_principal_df['longitude'] = pista_principal_df['longitude'].str.replace(',', '.').astype(float)
pista_principal_df['km_m'] = pista_principal_df['km_m'].str.replace(',', '.').astype(float)
print(pista_principal_df.head())
print(pista_principal_df.info())


          concessionaria  ano_do_pnv_snv    rodovia  km_m      sentido  \
0  AUTOPISTA FERNÃO DIAS            2007  BR-381/SP  22.6  Decrescente   
1  AUTOPISTA FERNÃO DIAS            2007  BR-381/SP  22.4  Decrescente   
2  AUTOPISTA FERNÃO DIAS            2007  BR-381/SP  22.2  Decrescente   
3  AUTOPISTA FERNÃO DIAS            2007  BR-381/SP  22.0  Decrescente   
4  AUTOPISTA FERNÃO DIAS            2007  BR-381/SP  21.8  Decrescente   

    latitude  longitude  
0 -22.999214 -46.518242  
1 -22.997770 -46.517069  
2 -22.996627 -46.515558  
3 -22.995466 -46.514080  
4 -22.994674 -46.512898  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255127 entries, 0 to 255126
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   concessionaria  255127 non-null  object 
 1   ano_do_pnv_snv  255127 non-null  int64  
 2   rodovia         255127 non-null  object 
 3   km_m            255127 non-null  float64
 4   sentido 

In [6]:
pista_principal_gdf = df2gdf_linestring(pista_principal_df, True)
print(pista_principal_gdf.head())
print(pista_principal_gdf.info())

  rodovia      sentido    concessionaria       km  ano_do_pnv_snv  \
0  092/PR  Decrescente  LITORAL PIONEIRO  324.090            2019   
1  151/PR    Crescente  LITORAL PIONEIRO  318.022            2019   
2  239/PR  Decrescente  LITORAL PIONEIRO   12.359            2019   
3  407/PR    Crescente  LITORAL PIONEIRO   17.990            2019   
4  408/PR    Crescente  LITORAL PIONEIRO   21.999            2019   

                                            geometry  
0  LINESTRING (-49.73240 -24.25181, -49.73472 -24...  
1  LINESTRING (-49.47956 -24.11651, -49.48566 -24...  
2  LINESTRING (-49.37777 -24.11891, -49.38658 -24...  
3  LINESTRING (-48.57749 -25.56884, -48.57285 -25...  
4  LINESTRING (-48.75282 -25.52557, -48.76237 -25...  
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   rodovia         99 non-null     object  
 1   se

In [7]:
def achar_estacao_mais_proxima(rodovias_gdf: gpd.GeoDataFrame, estacoes_gdf: gpd.GeoDataFrame, raio = 10000) -> gpd.GeoDataFrame:
    rodovias_gdf_copy = rodovias_gdf.copy()
    estacoes_gdf_copy = estacoes_gdf.copy()

    if rodovias_gdf_copy.crs is None:
        rodovias_gdf_copy = rodovias_gdf_copy.set_crs(epsg=4326)
    if estacoes_gdf_copy.crs is None:
        estacoes_gdf_copy = estacoes_gdf_copy.set_crs(epsg=4326)
    
    rodovias_gdf_metrico = rodovias_gdf_copy.to_crs(epsg=3857)
    estacoes_gdf_metrico = estacoes_gdf_copy.to_crs(epsg=3857)
    
    pontos_estacoes_proximas = []

    for _, rodovia in rodovias_gdf_metrico.iterrows():
        estacoes_proximas = estacoes_gdf_metrico[estacoes_gdf_metrico.geometry.distance(rodovia.geometry) <= raio]
        if not estacoes_proximas.empty:
            codigo_estacao = estacoes_proximas['codigo_estacao']
            # multipoint_estacoes = shapely.MultiPoint(estacoes_proximas.geometry.values)
        else:
            codigo_estacao = None
            # multipoint_estacoes = None  # Nenhuma estação dentro do raio
        
        # pontos_estacoes_proximas.append(multipoint_estacoes)
        pontos_estacoes_proximas.append(codigo_estacao)
    
    rodovias_gdf_copy["estacoes_proximas"] = pontos_estacoes_proximas
    return rodovias_gdf_copy.to_crs(rodovias_gdf_copy.crs)


rodovias_estacoes_gdf = achar_estacao_mais_proxima(pista_principal_gdf, inmet_gdf)
print(rodovias_estacoes_gdf.head())
print(rodovias_estacoes_gdf.info())
rodovias_estacoes_gdf.to_csv('rodo_estacoes.csv')

/node/guibatalhoti/antt_data_analysis/env/lib/python3.12/site-packages/shapely/measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)


  rodovia      sentido    concessionaria       km  ano_do_pnv_snv  \
0  092/PR  Decrescente  LITORAL PIONEIRO  324.090            2019   
1  151/PR    Crescente  LITORAL PIONEIRO  318.022            2019   
2  239/PR  Decrescente  LITORAL PIONEIRO   12.359            2019   
3  407/PR    Crescente  LITORAL PIONEIRO   17.990            2019   
4  408/PR    Crescente  LITORAL PIONEIRO   21.999            2019   

                                            geometry  \
0  LINESTRING (-49.73240 -24.25181, -49.73472 -24...   
1  LINESTRING (-49.47956 -24.11651, -49.48566 -24...   
2  LINESTRING (-49.37777 -24.11891, -49.38658 -24...   
3  LINESTRING (-48.57749 -25.56884, -48.57285 -25...   
4  LINESTRING (-48.75282 -25.52557, -48.76237 -25...   

                                   estacoes_proximas  
0  424     JOAQUIM TAVORA
696     JOAQUIM TAVORA
...  
1  478     CASTRO
928     CASTRO
1763    CASTRO
2...  
2                                               None  
3                           

/node/guibatalhoti/antt_data_analysis/env/lib/python3.12/site-packages/geopandas/io/file.py:612: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)
/node/guibatalhoti/antt_data_analysis/env/lib/python3.12/site-packages/fiona/collection.py:659: UserWarning: CSV driver silently converts datetime to string
  warnings.warn(


DriverIOError: Attempt to create csv layer (file) against a non-directory datasource.

In [8]:
import folium

m = folium.Map(location=[-15.788497, -47.879873], zoom_start=4)

#rodovias e estações com a mesma cor para melhor identificação
pista_principal_gdf.crs = "EPSG:4326"

folium.GeoJson(pista_principal_gdf, 
               name="Pista Principal", 
               style_function=lambda x: {'color': 'blue', 'weight': 5, 'opacity': 0.6},
               popup=folium.GeoJsonPopup(fields=['rodovia', 'concessionaria'], labels=True),
               tooltip=folium.GeoJsonTooltip(fields=['rodovia', 'concessionaria'], labels=True),
               zoom_on_click=True).add_to(m)

for _, rodovia in rodovias_estacoes_gdf.iterrows():
    if rodovia['estacoes_proximas'] is not None:
        for estacao in rodovia['estacoes_proximas']:
            x, y = inmet_gdf[inmet_gdf['codigo_estacao'] == estacao].geometry.x, inmet_gdf[inmet_gdf['codigo_estacao'] == estacao].geometry.y
            x, y = x.values[0], y.values[0]
            folium.CircleMarker(location=[y, x], radius=5, color='red').add_to(m)
        
folium.LayerControl().add_to(m)
m.save('mapa.html')


In [9]:
print(pista_principal_gdf.crs, inmet_gdf.crs)

EPSG:4326 EPSG:4326


Segunda forma de calcular

In [10]:
pista_principal_gdf.to_crs("EPSG:32633")
inmet_gdf.to_crs("EPSG:32633")

raio = 2
buffer_rodovia = pista_principal_gdf.copy()
buffer_rodovia['geometry'] = buffer_rodovia.buffer(raio * 1000)

estacoes_proximas = gpd.sjoin(buffer_rodovia, inmet_gdf, op='within')

pista_principal_gdf.to_crs("EPSG:4326")
inmet_gdf.to_crs("EPSG:4326")

estacoes_proximas.to_csv('estacoes_proximas.csv')

/tmp/ipykernel_1654536/2890774956.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  buffer_rodovia['geometry'] = buffer_rodovia.buffer(raio * 1000)


: 